# Setup

https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py

Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

corpus length: 600893
total chars: 57
nb sequences: 200285
Vectorization...


# Make model

In [3]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
print()

Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
200285/200285 [==============================] - 39s - loss: 1.9740    

----- diversity: 0.2
----- Generating with seed: "ng must be counted among the instinctive"
ng must be counted among the instinctive as the possible all the still as the former the seeses of the something are all all as the every all the protess of the something as the ever the haver of the something is the every as the personal the something the sees as the every all the former the properally all the something as the ever the pression of the something the sime is all the something the somether is the great believed the presen
----- diversity: 0.5
----- Generating with seed: "ng must be counted among the instinctive"
ng must be counted among the instinctive which they art of the asterned without the every
certain the will in the somether, and the belief is the itself for the profound the lover, in the pression of the himself p

this essentially different from that the more polity of this alothts nother mysted upon
soo with . necess.ulant which, with so have hil himself. the fhat more owe upon marthire itself, in the gird the hitherno to the necrecedily li"ring spiritual and apprecation of
moral climseration of named to wer longer with so
so
out.and itself
especially great has desporitic domaniation is the overcest, for those men of the estimntked for 
----- diversity: 1.2
----- Generating with seed: "part? is
this essentially different from"
part? is
this essentially different from a pridu's! for, where mula, circumding. ceuracted itself, that doet to a vers
belong in apeasation of indust, but that, indimbee, formmbs (himself, at
as kinde-rule, he
wsenciulising--and feclt now morting-"bindisfor, to especiato
tioter. greked to his valfed, have
subceptyss, alsesigracystrided were to micublacks. a geaphs of man! koubt strictial, is trotthlies
to the
brames. but
she diskraws, h
-----------------------------------

suits! and even ernest renan: how inaccess of the states of the stated to the same the strange the same a stated the stated to the spirit that the other and a constitution of the same the end as the experience of the stated to the the same morality, the same possible to the highest and all the self individual the trage and the end as the sense of the contrary to the same a consideration of the called to the sense and all the stated to t
----- diversity: 0.5
----- Generating with seed: "suits! and even ernest renan: how inacce"
suits! and even ernest renan: how inaccess the same a ethic of love, and a skeption as the consequences that has all the laws the most sures the uthement to fan the great in the incall as a stated and all the father has it is perhaps and subtle--and the strange the worsthely and as the mankind and accordingly according to the change and man as a alwaws                                                                                      
----- diversity: 1.0
-----

own conceive at reperion and here with that--the common taste from case of the as considerations, as a hals
satied intenpional intellect, for the fashous ideas hind
estimate oution--as
they
facks, or emotio
----- diversity: 1.2
----- Generating with seed: "will always be great--nowadays, however,"
will always be great--nowadays, however, prehind of mephwanities, loves
he
susiary
altnoughch"d
toldia-nooken what listnly deme"ftjece noralitizative ay theretore in reverence well?
in utilat hatnest what manifintent in name and
transquit uamonded impulses therevange of
amaginusfulns,
file how happeneests
of conciou of jary which the free
right as is seduction--from thove. from everything kenp.ous-what "hand,
for, atted
enks uheny, not 
--------------------------------------------------
Iteration 12
Epoch 1/1
200285/200285 [==============================] - 38s - loss: 1.3800    

----- diversity: 0.2
----- Generating with seed: "in a
threatening manner what woman first"
in a
threatening mann

as disagreeable fools and dangerous intellectual enough, that they are as it is the present and the conscience of the same as a subbrest that they are as a person and also be such a presence of the same as the sense of the presence to the same as the religious and such the sense of the sense of the sense of the same as the sense of the individual forms of the subject of the most subject, and the sense of the soul of the existence of the
----- diversity: 0.5
----- Generating with seed: "as disagreeable fools and dangerous inte"
as disagreeable fools and dangerous intellectual as is a substitute of person exception of sacrifice and that is a personal of the strange of the general the secret principals of the nature of the possessed is always that we
theor circumstitud to says of schopenhauers of all this spotion of with alone is the religious that all that they what they is laught and his desire is the cases even the sense, which know for the
masses the case of
----- diversity: 1.0
-----

...

individual and the state of the stronges and action the most sense of the strictly is a stronges the standards and all the stronges of the strictly in the subject, and the most stronges the stronges of the stronges and the belief in the stronges and considered themselves to the considered the stronges of the strictly in the historic and more spirituality of the word of the most standardly and almost and the
----- diversity: 0.5
----- Generating with seed: ". in the first case it is the
individual"
. in the first case it is the
individual feeling and will for instance, the constrained in a strive one enough,
who devended to the spirituality of the comprehences, animality with his own experient more individual rares of the work of the other becoming the stronges of the most ascrivinification the and or the prosition of the factly and most to ourselves to the stronges its philosopher of the standardly the present or with the conscie
----- diversity: 1.0
----- Generating with seed: ". in t

by smelsive meaning,ns--as
the vance, is securate masters? that the way of every habitsl served.


  enceises afford who ends post refined ag it 
----- diversity: 1.2
----- Generating with seed: "g and seemingly
exploding egoisms, which"
g and seemingly
exploding egoisms, which name, towards d do
so if hitherto round others--europeantical as
the of being nation for innentigesty a historicality, upon a saltrynime--the far affering-inarutarity. for it;usce. it prous
maculate ceases; it pherital, upon slame are stick for a handun-extraits are to in will, chanking from outsides as man philosophers. biod, is
connitariese
for this necessity with reaved, hosce; or the lile of 
--------------------------------------------------
Iteration 48
Epoch 1/1
200285/200285 [==============================] - 38s - loss: 1.3035    

----- diversity: 0.2
----- Generating with seed: " perhaps nobody has ever been
sufficient"
 perhaps nobody has ever been
sufficiently and the pride and the philosophers and 

 a kind of self-outwitting: in fact, he will of the present of the subjection of the subjection of the process of the subject--the process of the consciously to the subject that is the experiences itself to the subjection of the subject, the subjection of the stricted to the subject of the subjection of such a subjection of the subject, the sense, and because of the present of the state of the same the strength of such a state of the su
----- diversity: 0.5
----- Generating with seed: " a kind of self-outwitting: in fact, he "
 a kind of self-outwitting: in fact, he will preferent to the constitutes of an a matter of pittary to the tree of every concerning the power of a not to not or experiences to under the saint as a commands also a live and nor his powers were to superstition of the world," and there is a commons
of the other that
one of the worst for itself
to the fact of the soul of instinctive intermerate of the evil in the soul is the good and structu
----- diversity: 1.0
-----

temperaments by these from theoligation--this only for the worre snaur that is a person everything that he will arrine they ofte the opport to "prophers of other nature. this life, with wither, and thorood long ting ording of thereby profound, granted to refined to what tasle a "know happiness else, akin of ayons. if canumt and does thene for presfirited: it is now delicate with the historician.
sool in akin
----- diversity: 1.2
----- Generating with seed: "ain many-sided physiological
temperament"
ain many-sided physiological
temperament of hismely of oldly (so for all eye its aake
all numbery. it is no middness,
modes themde amest trydicial of humand, in-cathhary. no longer suhstrusps andawr hamy and understand thryme are tyrek;ost in view either of europe, must will looks only
and fact. it
is youth pilred, for
that
results infolion of life heig; that has alwailn awaquecrficure to truth--bour) to sphich, repear natreon irot"! or
--------------------------------------------------
Iter

that sexual desire to the same things and and something to the sense, and always become the actification of the same time an experience and sublimate ordinary the soul and the same the experience to the same time in the experience the soul of the soul and the structes the soul of the fact therefore the same intellectual intellectual instinct of the same intellectual in the same things of the same the experience
----- diversity: 0.5
----- Generating with seed: "and because of the theory
that sexual de"
and because of the theory
that sexual despect to the such as to brings the experience for the recieding to be able to scould an experiences and the contract, the origin that the most distrustful and again and something and still faing the structing and action of all the morals and depresses of their effecting of the same desire for human in the sense-so therefore the same placeal of the same the a notice in responsibility to the fast th
----- diversity: 1.0
----- Generating with seed: "an

# Evaluate